In [1]:
# Samuel Grant 2024

## Read failure events and make plots. 

### Based on PlotFailures.py. Better suited to notebook (so we don't have to load the data each time). 

In [2]:
# External libraries
import numpy as np
import pandas as pd
import awkward as ak
import uproot

# Internal
sys.path.append(os.path.abspath("../PyMacros"))
import Utils as ut
import PrintUtils as pr

In [3]:
# Parameters
PEs=100
layers="3"
particle="all"

In [4]:
# Persistent failure info; useful to know. 
failureFilePathNoTrackCuts = f"../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/failures_concise/failures_concise_all_{PEs}PEs{layers}Layers_no_track_cuts.csv"
print(f"\n---> Using info file {failureFilePathNoTrackCuts}.")
failureInfoNoTrackCuts_ = pd.read_csv(failureFilePathNoTrackCuts)
print(failureInfoNoTrackCuts_)

failureFilePathTrackCuts = f"../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/failures_concise/failures_concise_all_{PEs}PEs{layers}Layers_track_cuts.csv"
print(f"\n---> Using info file {failureFilePathTrackCuts}.")
failureInfoTrackCuts_ = pd.read_csv(failureFilePathTrackCuts)
print(failureInfoTrackCuts_)
print()

# Results info; useful to know. 
resultsFilePathNoTrackCuts = f"../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/results/results_all_{PEs}PEs{layers}Layers_no_track_cuts.csv"
print(f"\n---> Using info file {resultsFilePathNoTrackCuts}.")
resultsNoTrackCuts_ = pd.read_csv(resultsFilePathNoTrackCuts)
print(resultsNoTrackCuts_)

resultsFilePathTrackCuts = f"../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/results/results_all_{PEs}PEs{layers}Layers_track_cuts.csv"
print(f"\n---> Using info file {resultsFilePathTrackCuts}.")
resultsTrackCuts_ = pd.read_csv(resultsFilePathTrackCuts)
print(resultsTrackCuts_)

ineffNoTrackCuts = np.sum(resultsNoTrackCuts_["Failures"])/np.sum(resultsNoTrackCuts_["Total"])
ineffTrackCuts = np.sum(resultsTrackCuts_["Failures"])/np.sum(resultsTrackCuts_["Total"])

# Start output string 
resultStr = f"""
**********************************
PEs: {PEs}
layers: {layers}/4
particle: {particle}


Total inefficiency...
No track cuts: {np.sum(resultsNoTrackCuts_["Failures"])}/{np.sum(resultsNoTrackCuts_["Total"])} = {ineffNoTrackCuts*100:.5f}%
Track cuts: {np.sum(resultsTrackCuts_["Failures"])}/{np.sum(resultsTrackCuts_["Total"])} = {ineffTrackCuts*100:.5f}%
**********************************
"""

print(resultStr)


---> Using info file ../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/failures_concise/failures_concise_all_100PEs3Layers_no_track_cuts.csv.
                   tag  evtinfo.run  evtinfo.subrun  evtinfo.event
0      001205_00000000         1205               0           9297
1      001205_00000000         1205               0          64065
2      001205_00000000         1205               0          67462
3      001205_00000000         1205               0         217917
4      001205_00000000         1205               0         418461
...                ...          ...             ...            ...
25488  001205_00000554         1205            9913         427138
25489  001205_00000554         1205            9914          13005
25490  001205_00000554         1205            9914         259010
25491  001205_00000554         1205            9914         379347
25492  001205_00000554         1205            9914         387897

[25493 rows x 4 columns]

---> Using info fi

In [5]:
# Inspect a failure (also in TrackCuts)
# tag="001205_00000020"
run=1205
subrun=299 # 2263
event=79434 # 473431
filename="/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00023/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000000.root" #/exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00069/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000020.root"
with uproot.open(filename) as file:
    print(file)
    array_ = ut.GetData(file)
    print(array_)
    print(array_["evt"]["evtinfo.event"])
    # runCondition = ak.any(array_["evt"]["evtinfo.run"] == run, axis=0)
    # subrunCondition = ak.any(array_["evt"]["evtinfo.subrun"] == subrun, axis=0)
    # eventCondition = ak.any(array_["evt"]["evtinfo.event"] == event, axis=0)
    runCondition = array_["evt"]["evtinfo.run"] == run
    subrunCondition = array_["evt"]["evtinfo.subrun"] == subrun
    eventCondition = array_["evt"]["evtinfo.event"] == event
    array_ = array_[runCondition & subrunCondition & eventCondition] 
    pr.PrintNEvents(array_, masks_=[")

<ReadOnlyDirectory '/' at 0x7f7601ec6830>
[{evt: {'evtinfo.run': 1205, ...}, crv: {...}, trk: {...}, ...}, ..., {...}]
[331, 379, 759, 891, 1432, 2430, ..., 498161, 499019, 499859, 499895, 499971]
-------------------------------------------------------------------------------------
***** evt *****
evtinfo.run: 1205
evtinfo.subrun: 299
evtinfo.eventid: 79434
***** crv *****
crvcoincs.sectorType: [1, 2, 3]
crvcoincs.nLayers [4, 4, 4]
crvcoincs.angle: [-0.293, -6.18e-12, -0.374]
crvcoincs.pos.fCoordinates: ([-825, -337, -841], [4.77e+03, 4.62e+03, 4.92e+03], [-1.16e+03, -437, -1.2e+03])
crvcoincs.timeStart: [6.11e+04, 6.11e+04, 6.11e+04]
crvcoincs.time: [6.11e+04, 6.11e+04, 6.11e+04]
crvcoincs.PEs: [433, 464, 547]
crvcoincs.PEsPerLayer[4]: [[127, 87.3, 125, 94.3], [116, 115, 110, 123], [144, 139, 132, 133]]
crvcoincs.nHits: [19, 8, 16]
crvcoincsmc.pdgId: [-13, -13, -13]
crvcoincsmc.valid: [True, True, True]
crvcoincsmc.primaryE: [6.26e+03, 6.26e+03, 6.26e+03]
***** trk *****
kl.status: [1

In [6]:
# Internal libraries
sys.path.append(os.path.abspath("../PyMacros"))
import Utils as ut
import PrintUtils as pr
%run PlotFitUtils.ipynb
from Mu2eEAF import ReadData as rd 

In [7]:
'''
Get failure info
For each file, collect the events. 

This takes quite a while (2-3 minutes maybe).
I spent a full day trying to multithread this, it blows up my memory. 
'''
def GetFailures(fileList_, PEs=10, layers="3", particle="all", cut="singles"):

     # Collect failure info
    failureFilePath = f"../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/failures_concise/failures_concise_all_{PEs}PEs{layers}Layers_{cut}.csv"

    print(f"\n---> Using info file {failureFilePath}.")
    failureInfo_ = pd.read_csv(failureFilePath)

    if failureInfo_.empty:
        print("---> No failure info.")
        dummy_ = {}
        for field, branches_ in ut.allBranchNames_.items():
            dummy_[field] = ak.zip({name: ak.Array([]) for name in branches_})
        return ak.zip(dummy_)

    # Collect file list
    tags_ = list(set(failureInfo_["tag"]))
    
    # Extract the tag from the file name
    def ExtractTag(fileName):
        parts = fileName.split('.')
        if len(parts) > 1:
            return parts[-2]
        return None

    # Filter and sort files based on tags
    fileList_ = sorted(
        [file for file in fileList_ if ExtractTag(file) in tags_]
        , key=lambda file: tags_.index(ExtractTag(file))
    )

    # testing
    # tags_ = tags_[:1]
    # fileList_ = fileList_[:1]
    
    # Bug check
    if len(fileList_) == len(tags_):
        print("\n---> Collected and sorted failure file names.")
    else:
        raise Exception("\n---> len(fileList_) != len(tags_)")

    completedFiles = 0
    totalFiles = len(fileList_)

    print(f"\n---> Iterating through {len(fileList_)} files.\n") 

    # Master array to hold results
    data_ = ak.Array([])
    
    # Iterate through file list.
    for tag, fileName in zip(tags_, fileList_): 
    
        # Find failure events
        thisFailureInfo_ = failureInfo_[failureInfo_["tag"] == tag]
        outputStr = ( 
            f"\n--->\n" 
            f"fileName: {fileName}\n"
            f"tag: {tag}\n"
            f"failures:\n{thisFailureInfo_}\n"
            f"---"
        )
        if False: print(outputStr)
    
        # Read the file
        with uproot.open(fileName) as file: 
            
            # file = (fileName, quiet=True)
            # Get array
            thisData_ = ut.GetData(file)
        
            if False: print(f"\n---> Loaded corresponding data.\n{thisData_}")
                
            if False: print(f"\n---> Applying masks.")
        
            # Extract unique values from DataFrame
            runs_ = set(thisFailureInfo_["evtinfo.run"])
            subruns_ = set(thisFailureInfo_["evtinfo.subrun"])
            events_ = set(thisFailureInfo_["evtinfo.event"])
        
            # Construct masks
            runCondition = ak.any([thisData_["evt"]["evtinfo.run"] == value for value in runs_], axis=0)
            subrunCondition = ak.any([thisData_["evt"]["evtinfo.subrun"] == value for value in subruns_], axis=0)
            eventCondition = ak.any([thisData_["evt"]["evtinfo.event"] == value for value in events_], axis=0)
        
            # Apply masks
            thisData_ = thisData_[runCondition & subrunCondition & eventCondition]
        
            # Append to master array
            if False: print(f"\n---> Appending failures to master array.")
            data_ = ak.concatenate([data_, thisData_], axis=0)

            if False:
                print(thisFailureInfo_)
                ut.PrintNEvents(data_)
    
            completedFiles += 1
            percentComplete = (completedFiles / totalFiles) * 100
            
            print(f"\r---> Processed {fileName} ({percentComplete:.1f}% complete...)", end="") 

    inputEventList = failureInfo_["evtinfo.event"]
    outputEventList = ak.flatten(data_["evt"]["evtinfo.event"], axis=None)

    return data_

In [8]:
# Get file list
fileListPath="../Txt/FileLists/MDC2020aeOnExpData.txt"
def ReadFileList(fileListPath):
  with open(fileListPath, "r") as fileList_:
    lines = fileList_.readlines()
    lines = [line.strip() for line in lines]  # Remove leading/trailing whitespace
  return lines
fileList_=ReadFileList(fileListPath)
print(f"---> {len(fileList_)} files in list.")

---> 96 files in list.


In [ ]:
failures_no_track_cuts_ = GetFailures(fileList_, PEs=PEs, layers=layers, particle=particle, cut="no_track_cuts")
# ut.PrintNEvents(data_singles_, 1);


---> Using info file ../Txt/MDC2020ae/AugustCRVMeeting/FourthPass/concatenated/failures_concise/failures_concise_all_100PEs3Layers_no_track_cuts.csv.

---> Collected and sorted failure file names.

---> Iterating through 96 files.

---> Processed /exp/mu2e/data/users/sgrant/CRVSim/CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.000/11946817/00/00069/nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000020.root (7.3% complete...)

In [ ]:
failures_track_cuts_ = GetFailures(fileList_, PEs=PEs, layers=layers, particle=particle, cut="track_cuts")

In [ ]:
# Plot failure info

SimpleBarChart({"No track cuts" : len(failures_no_track_cuts_["evt"]["evtinfo.event"]), "Track cuts" : len(failures_track_cuts_["evt"]["evtinfo.event"]) }
         , ylabel="Failures", fout=f"../Images/MDC2020ae/Failures/FouthPass/bar_failures.png")


BarChart(ak.flatten(failures_no_track_cuts_["crv"]["crvcoincsmc.pdgId"], axis=None)
         , label_dict=particle_dict, title=f"No track cuts ({PEs} PEs/layer, {layers}/4 layers)",  ylabel="Unique particle species / failure", fout=f"../Images/MDC2020ae/Failures/FouthPass/bar_pdgid_no_tracks.png", percentage=False)

BarChart(ak.flatten(failures_track_cuts_["crv"]["crvcoincsmc.pdgId"], axis=None)
         , label_dict=particle_dict, title=f"Track cuts ({PEs} PEs/layer, {layers}/4 layers)", ylabel="Unique particle species / failure", fout=f"../Images/MDC2020ae/Failures/FouthPass/bar_pdgid_track_cuts.png", percentage=False)

particle_dict = {
        2212: 'proton',
        -211: 'pi-',
        -11: 'e+',
        11: 'e-',
        # "other": "other"
        # Add more entries as needed
    }

BarChartOverlay(data_=[ak.flatten(failures_no_track_cuts_["crv"]["crvcoincsmc.pdgId"], axis=None), ak.flatten(failures_track_cuts_["crv"]["crvcoincsmc.pdgId"], axis=None)]
                , label_dict=particle_dict
                , ylabel="Unique particle species / failure"
                , title=f"{PEs} PEs/layer, {layers}/4 layers"
                , fout=f"../Images/MDC2020ae/Failures/FouthPass/bar_pdgid_track_cuts_overlay.png"
                , percentage=False
                , label_= ["No track cuts", "Track cuts"])


Plot2D(x=ak.flatten(failures_no_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fZ"], axis=None)
       , y=ak.flatten(failures_no_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fX"], axis=None)
       , nbinsX=100, xmin=-8000, xmax=8000, nbinsY=100, ymin=-8000, ymax=8000
       , title="No track cuts", xlabel="KKInter Z [mm]", ylabel="KKInter X [mm]"
       , fout="../Images/MDC2020ae/Failures/ThirdPass/h2_ZX_no_track_cuts.png")

Plot2D(x=ak.flatten(failures_no_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fZ"], axis=None)
       , y=ak.flatten(failures_no_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fY"], axis=None)
       , nbinsX=100, xmin=-8000, xmax=8000, nbinsY=100, ymin=0, ymax=6000
       , title="No track cuts", xlabel="KKInter Z [mm]", ylabel="KKInter Y [mm]"
       , fout="../Images/MDC2020ae/Failures/ThirdPass/h2_ZY_no_track_cuts.png")

Plot2D(x=ak.flatten(failures_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fZ"], axis=None)
       , y=ak.flatten(failures_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fX"], axis=None)
       , nbinsX=100, xmin=-8000, xmax=8000, nbinsY=100, ymin=-8000, ymax=8000
       , title="Track cuts", xlabel="KKInter Z [mm]", ylabel="KKInter X [mm]"
       , fout="../Images/MDC2020ae/Failures/ThirdPass/h2_ZX_track_cuts.png")

Plot2D(x=ak.flatten(failures_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fZ"], axis=None)
       , y=ak.flatten(failures_track_cuts_["trkfit"]["klfit"]["pos"]["fCoordinates"]["fY"], axis=None)
       , nbinsX=100, xmin=-8000, xmax=8000, nbinsY=100, ymin=0, ymax=6000
       , title="No track cuts", xlabel="KKInter Z [mm]", ylabel="KKInter Y [mm]"
       , fout="../Images/MDC2020ae/Failures/ThirdPass/h2_ZY_track_cuts.png")

In [ ]:
# # What is more interesting is those that pass the singles cut, but not the track cut.
# # Although, this good be biased a bit because sometimes the track cuts generate an exception, while the singles do not!
# # Need to ID a file where this doesn't happen. (See ../Scripts/FindGoodFiles.sh)
# # 79 good files, written tags to ../Txt/MDC2020ae/GoodFiles/good_files.csv

# goodFileTags_ = ReadFileList("../Txt/MDC2020ae/GoodFiles/good_files.csv")

# # Make a dataframe for singles and singles_track_cuts using these files. 
# def CollectGoodFiles(goodFileTags_, PEs="10", layer="3", particle="all", cut="singles"):
#     baseDir=f"../Txt/MDC2020ae/failures_concise"
#     data_ = pd.DataFrame()
#     tasks = len(goodFileTags_)
#     completedTasks = 1
#     for i, goodFileTag in enumerate(goodFileTags_):
#         if i == 0: continue # skip header
#         goodFile = f"{baseDir}/{goodFileTag}/failures_concise_{particle}_{PEs}PEs{layer}Layers_{cut}.csv"
#         thisData_ = pd.read_csv(goodFile)
#         # Append data to the main DataFrame without resetting the index
#         if thisData_ is not None: 
#             # Add the tag to the first column
#             thisData_.insert(0, "Tag", goodFileTag)
#             # Concatenate
#             data_ = pd.concat([data_, thisData_], ignore_index=True)
#         completedTasks += 1
#         percentComplete = (completedTasks / tasks) * 100
#         print(f"\r---> Processed {goodFileTag} ({percentComplete:.1f}% complete...)", end="") 
#     return data_

# data_singles_ = CollectGoodFiles(goodFileTags_, PEs=10, cut="singles")
# data_singles_track_cuts_ = CollectGoodFiles(goodFileTags_, PEs=10, cut="singles_track_cuts")

# print(f"\n---> data_singles_:\n{data_singles_}")
# print(f"\n---> data_singles_track_cuts_:\n{data_singles_track_cuts_}")

In [ ]:
# # Now find entries unique to singles_track_cuts_ 
# # Merge DataFrames with an left join to get rows unique to singles_
# # I run the singles filter before the track cuts filter. 
# # This means that some events will pass the singles filter, and then be removed by the track cuts filter.
# # These events are unique to singles. 
# # We can look at the in the event display to see what happens in these cases. 

# # We need events which pass the singles cut but are then removed by the track cuts. 
# # These are actually not present in any of the failure samples, because these just contain failures. 
# # Unfortunately I do not put any marker on the ones removed by the tracker cuts, I just through them out. 

# # Merge
# data_merged_ = pd.merge(data_singles_, data_singles_track_cuts_, how="left", indicator=True)
# # Find unique to singles events
# unique_to_singles_ = data_merged_[data_merged_['_merge'] == 'left_only']
# # Drop the _merge column
# unique_to_singles_ = unique_to_singles_.drop('_merge', axis=1)
# print(unique_to_singles_)

In [ ]:
# Run it again?